# Getting all available Swiss supermarkets using the overpass Web API

## Overpass turbo query to get all available supermarkets in Switzerland

In [10]:
from pandas import json_normalize                      # Importiert die Funktion json_normalize zum Umwandeln von JSON-Daten in ein DataFrame
import requests                                        # Importiert das Modul requests für HTTP-Anfragen
import json                                            # Importiert das json-Modul zum Lesen und Schreiben von JSON-Dateien
import folium                                          # Importiert das Modul folium für die Erstellung interaktiver Karten

# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'         # Definiert die URL der Overpass-API (für OpenStreetMap-Abfragen)

# Overpass turbo query
query = f"""                                           
        [out:json];                                    
        area["ISO3166-1"="CH"][admin_level=2];         
        node ["shop"="supermarket"](area);             
        out;"""                                                                
    #    [out:json];                                    # Definiert das Ausgabeformat als JSON
    #    area["ISO3166-1"="CH"][admin_level=2];         # Wählt das Land Schweiz basierend auf ISO-Code und Verwaltungsebene 2
    #    node ["shop"="supermarket"](area);             # Wählt alle Knotenpunkte mit dem Tag "shop=supermarket" im gewählten Gebiet
    #    out;"""                                        # Gibt die Ergebnisse aus


# Web API request
r = requests.get(url, params={'data': query})          # Sendet die Overpass-Abfrage an die API und speichert die Antwort in 'r'
data = r.json()['elements']                            # Wandelt die Antwort in JSON um und extrahiert die Liste der Elemente

# Save data to file
with open('supermarkets.json', 'w') as json_file:      # Öffnet eine Datei zum Schreiben der Ergebnisse im JSON-Format
    json.dump(data, json_file)                         # Schreibt die JSON-Daten in die Datei 'supermarkets.json'

# Store data in data frame
df = json_normalize(data)                              # Normalisiert die JSON-Daten und konvertiert sie in ein pandas DataFrame
df.head(5)                                             # Zeigt die ersten fünf Zeilen des DataFrames an

,type,id,lat,lon,tags.addr:city,tags.addr:country,tags.addr:housenumber,tags.addr:postcode,tags.addr:street,tags.brand,...,tags.postid,tags.diet:local,tags.payment:account_cards,tags.diet:mediterranean,tags.diet:organic,tags.diet:seafood,tags.recycling:low_energy_bulbs,tags.opening_hours:cafe,tags.name:zh-Hans,tags.panoramax
0,node,33126515,47.155616,9.037915,Schänis,CH,32,8718,Biltnerstrasse,Spar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,36726161,47.226191,8.980329,Uznach,NaN,25,8730,Zürcherstrasse,Migros,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,39768209,47.225154,8.969868,Uznach,NaN,NaN,8730,NaN,Coop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,39947904,47.376732,8.542161,Zürich,CH,1,8001,Bahnhofbrücke,Coop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,48932835,47.375020,8.522895,Zürich,NaN,7,8004,Wengistrasse,Migros,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Plot supermarkets on map

In [ ]:
# Subset of supermarkets by brand
locations = df[["lat", "lon", "tags.brand", "tags.shop"]].loc[df["tags.brand"].isin(['Coop', 'Migros'])]   # Filtert Supermärkte, deren Marke Coop oder Migros ist
print(locations.head(5))                                                                                   # Gibt die ersten fünf Zeilen des gefilterten DataFrames aus

# Subsets
locations_coop = df[["lat", "lon", "tags.brand", "tags.shop"]].loc[df["tags.brand"].isin(['Coop'])]        # Erstellt Teilmenge nur mit Coop-Filialen
locations_migros = df[["lat", "lon", "tags.brand", "tags.shop"]].loc[df["tags.brand"].isin(['Migros'])]    # Erstellt Teilmenge nur mit Migros-Filialen

# Marker symbols
url_01 = 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-green.png'  # URL des grünen Markers für Coop
url_02 = 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-orange.png' # URL des orangen Markers für Migros

# Create map
map = folium.Map(location=[locations.lat.mean(), locations.lon.mean()],   # Erstellt eine Karte, zentriert auf dem geografischen Mittel der Standorte
                 zoom_start=8,                                            # Start-Zoomstufe 8 (Übersicht der Schweiz)
                 control_scale=True)                                      # Zeigt eine Massstabsleiste auf der Karte an

# Add lat/lon of Coop supermarkets
for i in range(0, len(locations_coop)):                                   # Schleife über alle Coop-Standorte
    folium.Marker(location=(locations_coop.iloc[i]['lat'],                # Marker-Position: Breitengrad
                            locations_coop.iloc[i]['lon']),               # Marker-Position: Längengrad
                  popup=locations_coop.iloc[i]['tags.brand'],             # Popup zeigt den Markennamen (Coop)
                  icon=folium.features.CustomIcon(url_01, icon_size=(16, 28))).add_to(map)  # Fügt grünen Marker mit definierter Grösse hinzu
    
# Add lat/lon of Migros supermarkets
for i in range(0, len(locations_migros)):                                 # Schleife über alle Migros-Standorte
    folium.Marker(location=(locations_migros.iloc[i]['lat'],              # Marker-Position: Breitengrad
                            locations_migros.iloc[i]['lon']),             # Marker-Position: Längengrad
                  popup=locations_migros.iloc[i]['tags.brand'],           # Popup zeigt den Markennamen (Migros)
                  icon=folium.features.CustomIcon(url_02, icon_size=(16, 28))).add_to(map)  # Fügt orangen Marker mit definierter Grösse hinzu

# Plot map
map                                                                         # Zeigt die interaktive Karte im Notebook an


         lat       lon tags.brand    tags.shop
1  47.226191  8.980329     Migros  supermarket
2  47.225154  8.969868       Coop  supermarket
3  47.376732  8.542161       Coop  supermarket
4  47.375020  8.522895     Migros  supermarket
6  47.491874  8.706448     Migros  supermarket


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')